In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.markers import MarkerStyle
from mplsoccer.pitch import Pitch
from mplsoccer import Pitch, VerticalPitch

from highlight_text import ax_text, fig_text
from PIL import Image
from mplsoccer import add_image
from urllib.request import urlopen

import requests

pd.set_option('display.max_columns', None)

In [ ]:
# Define the function to scrape shot data for a given matchId
def scrape_shots(mi):
    params = {'matchId': mi}
    response = requests.get('https://www.fotmob.com/api/matchDetails', params=params)
    data = response.json()
    shotmap = data['content']['shotmap']['shots']
    shots_df = pd.DataFrame(shotmap)
    shots_df['matchId'] = mi
    return shots_df

# Read the CSV containing the team names according to their respective Fotmob teamId
df_teamNameId = pd.read_csv(r"D:\FData\club_teams_logo\teams_name_and_id.csv")

# Define your list of matchIds (mi)
first_matchId = 4205343
match_ids = [first_matchId + i for i in range(380)]  

# Initialize an empty list to store individual DataFrames
all_shots_dfs = []

# Loop through each matchId, scrape the shots data, merge with team names, and append to the list
for fotmob_matchId in match_ids:
    shots_df = scrape_shots(fotmob_matchId)
    shots_df = shots_df.merge(df_teamNameId[['teamId', 'teamName']], on='teamId', how='left')
    
    # Adding the opposition team name column
    team1 = shots_df['teamName'].unique()[0]
    team2 = shots_df['teamName'].unique()[1]
    
    def get_opposite_teamName(team):
        if team == team1:
            return team2
        elif team == team2:
            return team1
        else:
            return None

    # Apply the function to create the new column
    shots_df['oppositeTeam'] = shots_df['teamName'].apply(get_opposite_teamName)
    
    # Append the DataFrame to the list
    all_shots_dfs.append(shots_df)

# Concatenate all DataFrames in the list into a single DataFrame
final_shots_df = pd.concat(all_shots_dfs, ignore_index=True)

# Cross cheking if all the data has extracted succesfully or not
print(len(final_shots_df))

In [ ]:
# saving the data in a csv
final_shots_df.to_csv("D:\FData\LaLiga\LaLiga2023_24_all_shots.csv")